## File summary
This is a testbench file for the code used in the other files.

The current file contains the following components:
* Loading model
* News-api testing
* Prompt template for summarising articles
* Combining model with template and testing





In [5]:
from dotenv import load_dotenv
load_dotenv()

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [17]:
import importlib


## Loading model

In [31]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama3-8b-8192", model_provider="groq")

## News api response

In [9]:
# code from news_search_tool.py

import requests
import os
query = 'IPL 2025'
url = "https://newsdata.io/api/1/latest"
params = {
    "q": query,
    "language": "en",
    "size": 3,
    "apikey":os.getenv("NEWS_API_KEY")
}
response = requests.get(url, params=params)
res = response.json()

In [11]:
res['results'][0]['description']

"Ian Bishop engaged in cheeky banter with his commentary colleagues during the ongoing Indian Premier League (IPL) and called some of them extremely 'pro-CSK'."

In [23]:
imported_module = importlib.import_module("news_search_tool")
importlib.reload(imported_module)
from news_search_tool import search_news
news_data = search_news('IPL 2025')

In [24]:
pp.pprint(news_data)

('When is Days Gone Remastered coming to PS5? Here’s the lowdown! - Days Gone '
 'Remastered is launching on PlayStation 5 on April 25, 2025, with enhanced '
 'graphics, lighting, and frame rates. It includes new game modes like Horde '
 'Assault, Permadeath, and Speedrun, along with an upgraded photo mode. The '
 'remastered version will be available for $49.99, with a $10 upgrade option '
 'for PS4 owners.\n'
 '<article separator>\n'
 "Max Verstappen Takes Hilarious 'Expensive Lawnmower' Jibe At Lando Norris "
 'After Dramatic Japanese GP Pit Moment | WATCH - Max Verstappen,Lando '
 'Norris,Japanese Grand Prix,F1 Pit Lane Incident,Verstappen Norris Close '
 'Call,Formula 1,McLaren,Red Bull Racing\n'
 '<article separator>\n'
 'IPL 2025: GT pacer Ishant Sharma handed one demerit point; fined 25 per cent '
 "of his match fees - Hyderabad Gujarat Titans' seasoned seamer Ishant Sharma "
 'has been handed one demerit point and fined per cent of his match fees after '
 'breaching the IPL Co

## Prompt template

#### "fernandoruiz/information_extraction_for_summarization"

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub

prompt = hub.pull("fernandoruiz/information_extraction_for_summarization")

# prompt template
'''
Please identify if the following news article is related to this topic : {topic}.
    [Start news article]:
    {article}
    [End news article]
    If the article is related to the topic, return a summary of the article related to the given topic.
    Answer in the most factual way possible. Only use content from the article.
    Please return the result in json using the following keys :
    "relevant"
    "who"
    "what"
    "where"
    "when"
    "why"
    If the article is not related return : "relevant" : false
    Answer:
'''

c:\Users\Admin\anaconda3\envs\langenv\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


#### Customised version of "fernandoruiz/information_extraction_for_summarization"

In [38]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Please identify if the following news articles are related to this topic : {topic}.
    [Start news article]:
    {article}
    [End news article]
    If any articles are related to the topic, use only those articles to return a summary related to the given topic.
    Answer in the most factual way possible. Only use content from the articles.

    Limit the answer only to the summary of the relevant article if available. Do not mention about the articles given as input.
    If the articles are not related return : "Articles not relevant to topic"
    Answer:''')

In [39]:
injected_prompt = prompt_template.invoke({"topic": "IPL 2025", "article": news_data})

In [28]:
pp.pprint(injected_prompt.text)

('\n'
 'Please identify if the following news articles are related to this topic : '
 'IPL 2025.\n'
 '    [Start news article]:\n'
 '    When is Days Gone Remastered coming to PS5? Here’s the lowdown! - Days '
 'Gone Remastered is launching on PlayStation 5 on April 25, 2025, with '
 'enhanced graphics, lighting, and frame rates. It includes new game modes '
 'like Horde Assault, Permadeath, and Speedrun, along with an upgraded photo '
 'mode. The remastered version will be available for $49.99, with a $10 '
 'upgrade option for PS4 owners.\n'
 '<article separator>\n'
 "Max Verstappen Takes Hilarious 'Expensive Lawnmower' Jibe At Lando Norris "
 'After Dramatic Japanese GP Pit Moment | WATCH - Max Verstappen,Lando '
 'Norris,Japanese Grand Prix,F1 Pit Lane Incident,Verstappen Norris Close '
 'Call,Formula 1,McLaren,Red Bull Racing\n'
 '<article separator>\n'
 'IPL 2025: GT pacer Ishant Sharma handed one demerit point; fined 25 per cent '
 "of his match fees - Hyderabad Gujarat Titans' 

## Combining prompt with model and testing

In [40]:
# injects the 'topic' and 'article' into the prompt
# injected_prompt = prompt.invoke({"topic": "IPL 2025", "article": news_data})

# invokes the model with the injected prompt 
answer = llm.invoke(injected_prompt)
answer

AIMessage(content="The articles are related to the topic IPL 2025. Here is a summary:\n\nGujarat Titans' pacer Ishant Sharma has been handed one demerit point and fined 25% of his match fees for breaching the IPL Code of Conduct during their match against Sunrisers Hyderabad.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 443, 'total_tokens': 505, 'completion_time': 0.051666667, 'prompt_time': 0.082855856, 'queue_time': 0.158819703, 'total_time': 0.134522523}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-2175bdf1-0a71-4ab9-a0d1-6a6e9abed62a-0', usage_metadata={'input_tokens': 443, 'output_tokens': 62, 'total_tokens': 505})

In [41]:
pp.pprint(answer.content)

('The articles are related to the topic IPL 2025. Here is a summary:\n'
 '\n'
 "Gujarat Titans' pacer Ishant Sharma has been handed one demerit point and "
 'fined 25% of his match fees for breaching the IPL Code of Conduct during '
 'their match against Sunrisers Hyderabad.')
